<a href="https://colab.research.google.com/github/FinSafe/MachineLearning/blob/main/Regression_Model_for_Predict_Total_Household_Income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library and Dataset

In [2]:
import pandas as pd
import gdown
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
import joblib

In [3]:
# Helper Function
def import_from_gdrive(id_file, file_name, sheet_name=None):
    link = 'https://drive.google.com/uc?id=' + id_file
    output_file = '/content/' + file_name
    gdown.download(link, output_file, quiet=False)

    df = pd.read_excel(output_file)
    return df

In [4]:
id_file = '1i8urlJHl90Mpq6eVIoOqsfJttx9F1HsA'
df = import_from_gdrive(id_file,'final_dataset_converted_rupiah.xlsx')

Downloading...
From: https://drive.google.com/uc?id=1i8urlJHl90Mpq6eVIoOqsfJttx9F1HsA
To: /content/final_dataset_converted_rupiah.xlsx
100%|██████████| 10.0M/10.0M [00:00<00:00, 61.7MB/s]


In [5]:
df = pd.read_excel('final_dataset_converted_rupiah.xlsx')

In [6]:
fix_columns =['Total Household Income',
              'Household Head Sex',
              'Household Head Age',
              'Household Head Marital Status',
              # 'Household Head Highest Grade Completed',
              'Household Head Highest Grade Completed (Simplified)',
              'Total Household Income',
              # 'Region',
              'Region Category',
              'Type of Building/House',
              'House Floor Area',
              'Number of bedrooms',
              'Electricity',
              'Tenure Status',
              'Type of Household',
              'Total Number of Family members',
              'Total number of family members employed',
              # 'Number of Car, Jeep, Van',
              # 'Number of Motorcycle/Tricycle',
              # 'Number of Television',
              # 'Number of Airconditioner',
              # 'Number of Personal Computer',
              # 'Number of Cellular phone',
              # 'Number of Refrigerator/Freezer',
              # 'Number of Washing Machine',
              'Number of Kids',
              'Number of Vehicles',
              'Number of Communication Devices',
              'Number of Electronics',
              ]

X =  df.loc[:, fix_columns]
X = X.loc[:,~X.columns.duplicated()]

# Mengubah tipe data int menjadi float
int_columns = X.select_dtypes(include=['int32', 'int64']).columns
X[int_columns] = X[int_columns].astype('float64')

y = df['Total Expenses']

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21930 entries, 0 to 21929
Data columns (total 69 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Total Household Income                               21930 non-null  float64
 1   Region                                               21930 non-null  object 
 2   Total Food Expenditure                               21930 non-null  float64
 3   Main Source of Income                                21930 non-null  object 
 4   Agricultural Household indicator                     21930 non-null  int64  
 5   Bread and Cereals Expenditure                        21930 non-null  float64
 6   Total Rice Expenditure                               21930 non-null  float64
 7   Meat Expenditure                                     21930 non-null  float64
 8   Total Fish and  marine products Expenditure          21930 non-nul

In [9]:
first_record_list_expenses = y.iloc[0].tolist()
first_record_list_expenses

171312833.062

In [47]:
first_record_list = X.iloc[0].tolist()
first_record_list

[182963970.185,
 'Female',
 49.0,
 'Single',
 'Degrees/Programs',
 'Category 0',
 'Single house',
 80.0,
 3.0,
 1.0,
 'Own or owner-like possession of house and lot',
 'Extended Family',
 4.0,
 1.0,
 1.0,
 1.0,
 2.0,
 3.0]

In [48]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21930 entries, 0 to 21929
Data columns (total 18 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Total Household Income                               21930 non-null  float64
 1   Household Head Sex                                   21930 non-null  object 
 2   Household Head Age                                   21930 non-null  float64
 3   Household Head Marital Status                        21930 non-null  object 
 4   Household Head Highest Grade Completed (Simplified)  21925 non-null  object 
 5   Region Category                                      21930 non-null  object 
 6   Type of Building/House                               21930 non-null  object 
 7   House Floor Area                                     21930 non-null  float64
 8   Number of bedrooms                                   21930 non-nul

In [49]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Pipeline preprocessing untuk fitur numerik
numeric_transformer = Pipeline(steps=[
    ('scaler', RobustScaler())  # Standarisasi
])
# Pipeline preprocessing untuk fitur kategorikal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())  # One-hot encoding
])

# Gabungkan pipeline preprocessing untuk fitur numerik dan kategorikal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

X = preprocessor.fit_transform(X)

joblib.dump(preprocessor, 'pipeline.pkl')

['pipeline.pkl']

In [26]:
TEST_SIZE = 0.2
RANDOM_STATE = 42
X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=TEST_SIZE, random_state=RANDOM_STATE)
n_cols = X_train.shape[1]

print("X_train shape : " , X_train.shape)
print("y_train shape : " , y_train.shape)
print("X_test  shape : " , X_test.shape)
print("y_test  shape : " , y_test.shape)

# X_train = X_train.toarray()  # Konversi SparseTensor menjadi numpy array
# X_test = X_test.toarray()    # Konversi SparseTensor menjadi numpy array

X_train shape :  (17544, 45)
y_train shape :  (17544,)
X_test  shape :  (4386, 45)
y_test  shape :  (4386,)


In [27]:
X_train[0]

array([-0.68890861, -0.625     ,  1.53846154, -1.        ,  0.        ,
        0.5       ,  0.        ,  1.        ,  1.        , -1.        ,
       -1.        ,  0.        ,  1.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ])

In [28]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [29]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01),input_shape=(X.shape[1],)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal', kernel_regularizer=l2(0.01)),
    tf.keras.layers.Dropout(0.2),  # Dropout layer
    tf.keras.layers.Dense(1, kernel_initializer='he_normal', activation='linear'),
])

# Kompilasi model

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.mean_absolute_error,
              metrics=["mae"],)

# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

# Latih model
model.fit(X_train, y_train, epochs=100, batch_size=128
          ,verbose=2,
          callbacks=[early_stopping])

# Evaluasi model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print('Mean Squared Error on Test Set:', mae)

# Prediksi
predictions = model.predict(X_test)

# Make predictions
y_preds = model.predict(X_test)

Epoch 1/100
138/138 - 2s - loss: 111691904.0000 - mae: 111691880.0000 - 2s/epoch - 15ms/step
Epoch 2/100
138/138 - 0s - loss: 61584292.0000 - mae: 61584124.0000 - 438ms/epoch - 3ms/step
Epoch 3/100
138/138 - 0s - loss: 23468126.0000 - mae: 23467906.0000 - 441ms/epoch - 3ms/step
Epoch 4/100
138/138 - 0s - loss: 21329160.0000 - mae: 21328920.0000 - 445ms/epoch - 3ms/step
Epoch 5/100
138/138 - 0s - loss: 20275534.0000 - mae: 20275300.0000 - 435ms/epoch - 3ms/step
Epoch 6/100
138/138 - 0s - loss: 19820806.0000 - mae: 19820566.0000 - 465ms/epoch - 3ms/step
Epoch 7/100
138/138 - 0s - loss: 19569428.0000 - mae: 19569176.0000 - 438ms/epoch - 3ms/step
Epoch 8/100
138/138 - 0s - loss: 19304172.0000 - mae: 19303924.0000 - 500ms/epoch - 4ms/step
Epoch 9/100
138/138 - 0s - loss: 19359028.0000 - mae: 19358778.0000 - 466ms/epoch - 3ms/step
Epoch 10/100
138/138 - 0s - loss: 19246836.0000 - mae: 19246592.0000 - 453ms/epoch - 3ms/step
Epoch 11/100
138/138 - 0s - loss: 19265684.0000 - mae: 19265440.0000 

# Result

In [30]:
# Epoch 44/100
# 138/138 - 1s - loss: 19111542.0000 - mae: 19111300.0000 - 669ms/epoch - 5ms/step
# Mean Squared Error on Test Set: 17637802.0

# Save Model

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 256)               11776     
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [32]:
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [33]:
model.save('saved_model/')

In [50]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.5
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.7.0
anyio                            3.7.1
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array_record                     0.5.1
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.1.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.15.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bidict                           0.23.1